In [1]:
import numpy as np
from keras_vggface.vggface import VGGFace
from keras.models import Model
from keras.layers import Flatten, Dense

Using TensorFlow backend.


In [2]:
vgg_model = VGGFace(include_top=False, input_shape=(128, 128, 3))


In [ ]:
vgg_model.summary()

In [3]:
for layer in vgg_model.layers:
    layer.trainable = False

In [4]:
last_layer = vgg_model.get_layer('pool5').output
print(last_layer)

Tensor("pool5/MaxPool:0", shape=(?, 4, 4, 512), dtype=float32)


In [22]:
x = Flatten()(last_layer)
x = Dense(1024, activation='relu')(x)
x = Dense(768, activation='relu')(x)
out = Dense(735, activation='softmax')(x)

custom_vgg_model = Model(vgg_model.input, out)

In [24]:
custom_vgg_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [25]:
custom_vgg_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 128, 128, 64)      1792      
_________________________________________________________________
conv1_2 (Conv2D)             (None, 128, 128, 64)      36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 64, 64, 64)        0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 64, 64, 128)       73856     
_________________________________________________________________
conv2_2 (Conv2D)             (None, 64, 64, 128)       147584    
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 32, 32, 128)       0         
__________

In [8]:
from keras.preprocessing.image import ImageDataGenerator

In [9]:
train_generator = ImageDataGenerator(
    vertical_flip=True,
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
)

val_generator = ImageDataGenerator(
    vertical_flip=True
)

In [23]:
train_flow = train_generator.flow_from_directory(
    '/home/pexea12/repos/face/face-core/data/train',
    target_size=(128, 128),
    batch_size=64,
)

val_flow = val_generator.flow_from_directory(
    '/home/pexea12/repos/face/face-core/data/validation',
    target_size=(128, 128),
    batch_size=64,
)

Found 46000 images belonging to 735 classes.
Found 709 images belonging to 24 classes.


In [26]:
custom_vgg_model.fit_generator(
    train_flow,
    steps_per_epoch=700,
    epochs=3,
    validation_data=val_flow,
    validation_steps=32
)

Epoch 1/3
699/700 [============================>.] - ETA: 2s - loss: 16.0932 - acc: 0.0015

ValueError: Error when checking target: expected dense_5 to have shape (None, 735) but got array with shape (64, 24)